Doc2Vec을 반드시 텍스트 데이터에 이용해야 하는 것은 아닙니다. Sequential 한 아이템으로 서술할 수 있는 임의의 객체면 모두 Doc2Vec을 이용하여 임베딩 벡터를 학습할 수 있습니다. 

예를 들어 영화에는 배우가 casting order 순으로 기록되어 있습니다. 이 정보를 이용하여 캐스팅이 유사한 영화를 구할 수 있습니다

In [12]:
import pickle

id2movie_fname = '../../../data/sample_naver_movie/navermovie_info_idx2moviename.pkl'
id2actor_fname = '../../../data/sample_naver_movie/navermovie_info_idx2actorname.pkl'
casting_fname =  '../../../data/sample_naver_movie/navermovie_casting.txt'

with open(id2actor_fname, 'rb') as f:
    idx2actor = pickle.load(f)
    
with open(id2movie_fname, 'rb') as f:
    idx2movie = pickle.load(f)    
    
movie2idx = {name:idx for idx, name in idx2movie.items()}
actor2idx = {name:idx for idx, name in idx2actor.items()}

영화 아이디를 document tags로, 배우를 단어로 취급합니다. Bag of Words와 비슷하게 Bag of Actors라는 이름의 클래스를 만들었습니다. 

    words = actors.split()
    
으로 만들어서 TaggedDocument의 words로 입력합니다

In [13]:
from gensim.models.doc2vec import TaggedDocument

class BagOfActors:
    def __init__(self, fname):
        self.fname = fname
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for row in f:
                movie_idx, actors = row.replace('\n','').split('\t')
                words = actors.split()
                if len(words) <= 1:
                    continue
                yield TaggedDocument(words = words, tags = ['MOVIE_ID = %s' % movie_idx])
        
        
bag_of_actors = BagOfActors(casting_fname)

for num, doc in enumerate(bag_of_actors):
    print(doc, '\n')
    if num > 3: break

TaggedDocument(['7995', '4009'], ['MOVIE_ID = 22682']) 

TaggedDocument(['154441', '12571'], ['MOVIE_ID = 25882']) 

TaggedDocument(['6852', '96763', '3107', '7067', '39104'], ['MOVIE_ID = 29394']) 

TaggedDocument(['187973', '339348'], ['MOVIE_ID = 120378']) 

TaggedDocument(['20842', '46829', '242523'], ['MOVIE_ID = 56605']) 



영화 배우는 한 번만 등장할 수도 있기 때문에 min_count=1로 하여 한 번이라도 등장한 배우는 이용하기로 합니다. 


In [17]:
from pprint import pprint
from gensim.models import Doc2Vec

doc2vec_model = Doc2Vec(bag_of_actors, min_count=1)

In [18]:
print(type(doc2vec_model.docvecs.doctags))
pprint(list(doc2vec_model.docvecs.doctags.items())[:3])

<class 'dict'>
[('MOVIE_ID = 72052', Doctag(offset=20396, word_count=16, doc_count=1)),
 ('MOVIE_ID = 90476', Doctag(offset=64322, word_count=5, doc_count=1)),
 ('MOVIE_ID = 73965', Doctag(offset=19230, word_count=9, doc_count=1))]


영화가 임베딩이 되었고, 비슷한 영화를 검색할 수 있는지 확인해보았습니다. 이제 영화 아이디를 이름으로 치환하여 비슷한 영화들이 무엇인지를 확인해봅시다

In [19]:
doc2vec_model.docvecs.most_similar(1, topn=3)

[('MOVIE_ID = 80980', 0.9521088600158691),
 ('MOVIE_ID = 124460', 0.942322313785553),
 ('MOVIE_ID = 65608', 0.9381561279296875)]

In [20]:
def movie_id2name(similar):
    idx = similar[0].split(' = ')[1]
    return (idx2movie.get(idx, 'unknown'), similar[1])

In [21]:
for similar in doc2vec_model.docvecs.most_similar(2, topn=3):
    print(movie_id2name(similar))

('아트풀 다저스', 0.8974316716194153)
('이것이 사랑이다', 0.89439457654953)
('단스', 0.8715944290161133)


송강호가 나왔던 영화 '관상'의 아이디를 찾기 위해서 movie2idx의 (key, value)를 for loop으로 돌면서 아이디를 찾았습니다. 영화 이름에 관상이 들어갔던 영화가 한 개 있었고, 아이디가 93728임을 확인합니다. 

In [22]:
for movie, idx in movie2idx.items():
    if '관상' in movie:
        print(movie, idx)

관상 93728


관상과 출연배우가 비슷한 영화를 찾았으나, 영화로는 잘 나오지 않았습니다. 사실 영화 자체가 배우 정보만 가지고 임베딩이 되기는 어렵습니다. 왜냐면 배우가 많아야 몇 십 편의 영화에 겨우 등장하며, 우리는 주연배우들에 의해서 영화 배우가 비슷하다고 판단하겠지만, 이 연습코드에서는 모든 배우의 중요도가 동일하였기 때문입니다. 

이 예시에서 한 가지 알 수 있는 것은, 임베딩을 학습하고 싶은 객체들을 잘 표현할 수 있는 데이터가 있어야 한다는 것입니다. 단순히 [영화 | 배우1, 배우2, ... ] 정보만으로는 임베딩이 어려울 수 있습니다

만약 주연 배우들의 힘을 더 실어주고 싶다면, 배우1이 4배, 배우2가 3배 더 중요하다면, 

    영화1 | 배우1 배우1 배우1 배우1 배우2 배우2 배우2 배우3 ...
    
와 같이 배우를 복제하고, Doc2Vec모델의 argument window를 크게 잡아줄 수 있습니다. 물론 임시방편인 방법입니다. 

In [23]:
for similar in doc2vec_model.docvecs.most_similar('MOVIE_ID = 93728', topn=30):
    print(movie_id2name(similar))

('데스퍼레이트 레미디스', 0.8805059790611267)
('펀치', 0.8799708485603333)
('감옥정사', 0.8627691268920898)
('테레즈', 0.8627644777297974)
('디스코 피그', 0.8596412539482117)
('드래그 미 투 헬', 0.8571529388427734)
('크리스마스', 0.8527121543884277)
('썸남썸녀', 0.851172924041748)
('코프스 행스 인 더 웹', 0.8508224487304688)
('믿거나 말거나 - 2차 TV 시리즈', 0.8479501008987427)
('독스 오브 헬', 0.8460571765899658)
('유리화', 0.8452415466308594)
('명왕계획 제오라이마', 0.844073474407196)
('스파이 인 노스코리아', 0.8428146839141846)
('블라인드 로맨스', 0.8427276611328125)
('빙고 봉고', 0.8415997624397278)
('노콘 키드~우리들의 게임사~', 0.8414942622184753)
('황제의 무덤', 0.8408210277557373)
('오델로', 0.8386428952217102)
('중2병이라도 사랑이 하고 싶어! 극장판', 0.8368615508079529)
('라우트로스', 0.8347976803779602)
('유령', 0.8343879580497742)
('시보', 0.8330366611480713)
('백년해로외전', 0.8300822973251343)
('고독한 방랑자의 전설', 0.8298344612121582)
('올림포스 가디언', 0.828048586845398)
('윈터 인 더 블러드', 0.8274933099746704)
('새벽', 0.8269795179367065)
('오 헨리 단편집', 0.8241780996322632)
('오퍼레이션 필름메이커', 0.8240978717803955)


배우는 혹시 임베딩이 잘 되었는지 살펴보겠습니다

In [24]:
actor_names = ['송강호', '톰 크루즈', '안성기', '이선균', '장국영', '장쯔이']
for actor, idx in actor2idx.items():
    for actor_name in actor_names:
        if actor_name in actor:
            print(actor_name, idx)

톰 크루즈 1558
안성기 1843
장쯔이 2372
송강호 1824
이선균 8017
장국영 1945


In [25]:
doc2vec_model.most_similar('1843')

[('1909', 0.9643524885177612),
 ('5045', 0.9560308456420898),
 ('3598', 0.95563805103302),
 ('5346', 0.9524948000907898),
 ('5933', 0.951680064201355),
 ('40908', 0.9511442184448242),
 ('2805', 0.9500079154968262),
 ('8963', 0.9478958249092102),
 ('3638', 0.9465739727020264),
 ('7277', 0.9456812739372253)]

In [26]:
def actor_id2name(similar):
    idx = similar[0]
    return (idx2actor.get(idx, 'unknown'), similar[1])

In [27]:
print('탐 크루즈\n')
for similar in doc2vec_model.most_similar('1558'):
    print(actor_id2name(similar))

탐 크루즈

('우마 서먼', 0.8335953950881958)
('마이클 더글라스', 0.829007089138031)
('라다 미첼', 0.8281677961349487)
('숀 펜', 0.8270123600959778)
('로렌 바콜', 0.8250910639762878)
('지나 데이비스', 0.8240168690681458)
('안젤리나 졸리', 0.8239791989326477)
('셀마 헤이엑', 0.8236217498779297)
('줄리앤 필립스', 0.8225073218345642)
('케이트 넬리건', 0.8223050832748413)


탐크루즈와 다르게 송강호는 한국배우들이 비슷한 출연을 했음을 확인할 수 있습니다

In [28]:
print('송강호\n')
for similar in doc2vec_model.most_similar('1824'):
    print(actor_id2name(similar))

송강호

('손예진', 0.9210221767425537)
('안선영', 0.9146535992622375)
('이영은', 0.9056206941604614)
('김지수', 0.9032304883003235)
('임원희', 0.9029457569122314)
('박희순', 0.9024836421012878)
('이승연', 0.9023244976997375)
('이종수', 0.89962238073349)
('윤소이', 0.8979786038398743)
('박주미', 0.8954672813415527)


안성기 배우는 연기 생활의 경력이 많기 때문에 오래전에 활동하셨던 김무생, 유인촌, 신구 등의 배우와 유사성이 있음을 확인할 수 있습니다. 영화는 학습이 잘 되지 않았지만, 배우는 아주 조금은 학습이 되었음을 확인할 수 있습니다. 

In [29]:
print('안성기\n')
for similar in doc2vec_model.most_similar('1843'):
    print(actor_id2name(similar))

안성기

('이영하', 0.9643524885177612)
('이대근', 0.9560308456420898)
('김창숙', 0.95563805103302)
('이덕화', 0.9524948000907898)
('오지명', 0.951680064201355)
('허진', 0.9511442184448242)
('백일섭', 0.9500079154968262)
('전양자', 0.9478958249092102)
('김인문', 0.9465739727020264)
('김무생', 0.9456812739372253)


젊은 배우 이선균을 검색하면 이러한 경향을 좀 더 확인할 수 있습니다

In [30]:
print('이선균\n')
for similar in doc2vec_model.most_similar('8017'):
    print(actor_id2name(similar))

이선균

('안재욱', 0.8800215721130371)
('김성수', 0.8669286370277405)
('김석훈', 0.8609542846679688)
('이영은', 0.8478140830993652)
('박희순', 0.8423821926116943)
('데니스 리어리', 0.8401180505752563)
('정진', 0.8380500078201294)
('소지섭', 0.8375609517097473)
('고창석', 0.8330643177032471)
('강선숙', 0.8314653038978577)


홍콩 배우 장국영을 검색하니 다른 홍콩 배우들이 등장함을 볼 수 있습니다

In [31]:
print('장국영\n')
for similar in doc2vec_model.most_similar('1945'):
    print(actor_id2name(similar))
    

장국영

('왕정', 0.9493590593338013)
('정유령', 0.9470380544662476)
('오가려', 0.9458274841308594)
('구숙정', 0.9413948059082031)
('여명', 0.9396499395370483)
('이자웅', 0.9392326474189758)
('유덕화', 0.9386542439460754)
('묘교위', 0.9385079145431519)
('양조위', 0.9351979494094849)
('주문건', 0.9348886609077454)


In [32]:
print('장쯔이\n')
for similar in doc2vec_model.most_similar('2372'):
    print(actor_id2name(similar))

장쯔이

('리지', 0.8983218669891357)
('노관정', 0.8904310464859009)
('전준', 0.8893892168998718)
('이연걸', 0.8878319263458252)
('금성무', 0.8873013257980347)
('오기륭', 0.885164201259613)
('진혜림', 0.8774100542068481)
('장학우', 0.8734366297721863)
('양쟁', 0.8718304634094238)
('매리 스튜어트 매스터슨', 0.8705610036849976)
